![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [97]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Import any additional modules and start coding below

In [98]:
rental_data = pd.read_csv('rental_info.csv')
# Display the first few rows of the dataset
print(rental_data.head())# Check for missing values

                 rental_date                return_date  amount  release_year  \
0  2005-05-25 02:54:33+00:00  2005-05-28 23:40:33+00:00    2.99        2005.0   
1  2005-06-15 23:19:16+00:00  2005-06-18 19:24:16+00:00    2.99        2005.0   
2  2005-07-10 04:27:45+00:00  2005-07-17 10:11:45+00:00    2.99        2005.0   
3  2005-07-31 12:06:41+00:00  2005-08-02 14:30:41+00:00    2.99        2005.0   
4  2005-08-19 12:30:04+00:00  2005-08-23 13:35:04+00:00    2.99        2005.0   

   rental_rate  length  replacement_cost                special_features  \
0         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
1         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
2         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
3         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
4         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   

   NC-17  PG  PG-13  R  amount_2  length_2  rental_rate_

In [99]:
rental_data.info()


<class 'pandas.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rental_date       15861 non-null  str    
 1   return_date       15861 non-null  str    
 2   amount            15861 non-null  float64
 3   release_year      15861 non-null  float64
 4   rental_rate       15861 non-null  float64
 5   length            15861 non-null  float64
 6   replacement_cost  15861 non-null  float64
 7   special_features  15861 non-null  str    
 8   NC-17             15861 non-null  int64  
 9   PG                15861 non-null  int64  
 10  PG-13             15861 non-null  int64  
 11  R                 15861 non-null  int64  
 12  amount_2          15861 non-null  float64
 13  length_2          15861 non-null  float64
 14  rental_rate_2     15861 non-null  float64
dtypes: float64(8), int64(4), str(3)
memory usage: 1.8 MB


In [100]:
rental_data['rental_date']=pd.to_datetime(rental_data['rental_date'])
rental_data['return_date']=pd.to_datetime(rental_data['return_date'])
rental_data['rental_length_days']=(rental_data['return_date']-rental_data['rental_date']).dt.days
# Check the distribution of rental lengths
print(rental_data['rental_length_days'].describe())

count    15861.000000
mean         4.525944
std          2.635108
min          0.000000
25%          2.000000
50%          5.000000
75%          7.000000
max          9.000000
Name: rental_length_days, dtype: float64


In [101]:
rental_data["deleted_scenes"] =  np.where(rental_data["special_features"].str.contains("Deleted Scenes"), 1,0)
rental_data["behind_the_scenes"] =  np.where(rental_data["special_features"].str.contains("Behind the Scenes"), 1,0)
print(rental_data[["deleted_scenes", "behind_the_scenes"]].head())

   deleted_scenes  behind_the_scenes
0               0                  1
1               0                  1
2               0                  1
3               0                  1
4               0                  1


In [102]:
X=rental_data.drop(columns=["rental_length_days", "rental_date", "return_date", "special_features"])
y=rental_data["rental_length_days"]
print(X.head())

   amount  release_year  rental_rate  length  replacement_cost  NC-17  PG  \
0    2.99        2005.0         2.99   126.0             16.99      0   0   
1    2.99        2005.0         2.99   126.0             16.99      0   0   
2    2.99        2005.0         2.99   126.0             16.99      0   0   
3    2.99        2005.0         2.99   126.0             16.99      0   0   
4    2.99        2005.0         2.99   126.0             16.99      0   0   

   PG-13  R  amount_2  length_2  rental_rate_2  deleted_scenes  \
0      0  1    8.9401   15876.0         8.9401               0   
1      0  1    8.9401   15876.0         8.9401               0   
2      0  1    8.9401   15876.0         8.9401               0   
3      0  1    8.9401   15876.0         8.9401               0   
4      0  1    8.9401   15876.0         8.9401               0   

   behind_the_scenes  
0                  1  
1                  1  
2                  1  
3                  1  
4                  1  


In [103]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)


In [104]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [105]:
lasso = Lasso(alpha=0.1, random_state=9)    
lasso.fit(X_train, y_train)
lasso_coefficients = lasso.coef_
print("Lasso Coefficients:", X_train.columns[lasso_coefficients != 0])


Lasso Coefficients: Index(['amount', 'replacement_cost', 'amount_2', 'length_2', 'rental_rate_2'], dtype='str')


In [106]:
X_train_lasso = X_train_scaled[:, lasso_coefficients > 0]
X_test_lasso = X_test_scaled[:, lasso_coefficients > 0]


In [109]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

models={
    "Linear Regression": LinearRegression(),'Lasso Regression': Lasso(),'Ridge Regression': Ridge(), 'Random Forest Regressor': RandomForestRegressor(random_state=9), 'Gradient Boosting Regressor': GradientBoostingRegressor(random_state=9)}

In [110]:
ols=LinearRegression()
ols.fit(X_train_lasso, y_train)   
y_pred_ols = ols.predict(X_test_lasso)
mse_ols = mean_squared_error(y_test, y_pred_ols)
print(f"OLS MSE: {mse_ols}")

OLS MSE: 4.812297241276237


In [112]:
param_dist = {'n_estimators': np.arange(1,1001,1),
          'max_depth':np.arange(1,101,1)}   

In [113]:
rf=RandomForestRegressor(random_state=9)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred)
print(f"Random Forest MSE: {mse}")


Random Forest MSE: 2.030141907417274


In [114]:
from sklearn.model_selection import RandomizedSearchCV
random_search_rf = RandomizedSearchCV(rf, param_distributions=param_dist,cv=5,random_state=9)
random_search_rf.fit(X_train, y_train)
best_rf = random_search_rf.best_params_
print(f"Best Random Forest Parameters: {best_rf}")


Best Random Forest Parameters: {'n_estimators': np.int64(935), 'max_depth': np.int64(56)}


In [115]:
best_rf_model = RandomForestRegressor(n_estimators=best_rf['n_estimators'], max_depth=best_rf['max_depth'], random_state=9)
best_rf_model.fit(X_train, y_train)
y_pred_rf = best_rf_model.predict(X_test)
mse_rf_tuned = mean_squared_error(y_test, y_pred_rf)
print(f"Random Forest Tuned MSE: {mse_rf_tuned}")

Random Forest Tuned MSE: 2.024252714220438


In [116]:
from sklearn.tree import DecisionTreeRegressor
dtr=DecisionTreeRegressor(max_depth=best_rf['max_depth'], random_state=9)
dtr.fit(X_train, y_train)
y_pred_dtr = dtr.predict(X_test)
mse_dtr = mean_squared_error(y_test, y_pred_dtr)
print(f"Decision Tree MSE: {mse_dtr}")

Decision Tree MSE: 2.1675004952579413


In [117]:
from sklearn.ensemble import GradientBoostingRegressor
gbr=GradientBoostingRegressor(random_state=9)
gbr.fit(X_train, y_train)
y_pred_gbr = gbr.predict(X_test)
mse_gbr = mean_squared_error(y_test, y_pred_gbr)
print(f"Gradient Boosting MSE: {mse_gbr}")

Gradient Boosting MSE: 2.4253464800253557


In [118]:
best_model=rf
best_mse=mse_rf